# Sentinel Hub Feature Info Service (FIS)

A statistical summary of satellite indices can be requested via the FIS feature. Examples of extracting time-series statistics data of Sentinel-2 and Sentinel-5P indices are provided.<br>

**Note: Cloud mask values have been included along with the obtained statistics values of Sentinel-2 indices in the following examples.**


In [11]:
# import libraries
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import geopandas as gpd


from sentinelhub import FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, \
    DataCollection, HistogramType, bbox_to_dimensions
from sentinelhub.time_utils import parse_time
from sentinelhub import DataCollection

# sentinel hub configurations
For the instance ID

In [12]:
from sentinelhub import SHConfig


INSTANCE_ID = '3704e2d2-43dc-4f34-a563-13ec19dff111'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

In [13]:
def fis_data_to_dataframe(fis_data):
    """ Creates a DataFrame from list of FIS responses
    """
    COLUMNS = ['channel', 'date', 'min', 'max', 'mean', 'stDev']
    data = []

    for fis_response in fis_data:
        for channel, channel_stats in fis_response.items():
            for stat in channel_stats:
                row = [int(channel[1:]), parse_time(stat['date'])]

                for column in COLUMNS[2:]:
                    row.append(stat['basicStats'][column])

                data.append(row)

    return pd.DataFrame(data, columns=COLUMNS).sort_values(['channel', 'date'])

In [14]:
# convert to clean df ready for analysis
def clean_df (json_stats):
    raw_df_stats = fis_data_to_dataframe(json_stats).reset_index(drop=True)
    index_df = raw_df_stats[raw_df_stats.channel ==0]
    cloud_df = raw_df_stats[raw_df_stats.channel ==1]
    cloud_df = cloud_df.drop(['channel'], axis=1).add_suffix('_clm')
    merged_df = pd.merge(index_df, cloud_df, left_on ='date', right_on = 'date_clm', how='outer')
    clean_merged_df = merged_df.drop(['channel', 'date_clm'], axis=1)
    return clean_merged_df

## Parameters of FIS request


* `layer` -   name of the layer defined in Sentinel Hub Configurator.
* `geometry_list` - list of geometry objects (BBox or Geometry), statistics will be calculated for each of them separately
* `time` - statistics will be calculated for each acquisition in the give time interval separately 
* `resolution` - spatial resolution on which to calculate statistics 
* `data_folder` - optional parameter for specifying location where the data should be saved locally 
* `bins` - The number of bins (a positive integer) in the histogram. When this parameter is absent, no histogram is computed.
* `histogram_type` - ways of dividing values into bins currently supported: EQUIDISTANT, EQUALFREQUENCY, or STREAMING

## Sentinel Hub Configuration

For the Sentinel-5P layer NO2, refer to **FIS-request** configuration instance

In [15]:
# read single_region geojson in geopandas
single_region = gpd.read_file("Metropolitan Manila.geojson")
single_region_geometry = Geometry(single_region.geometry.values[0], crs=CRS.WGS84)

In [16]:
# initialize request to obtain basic stats and histogram values
# warning if the timeline is to long there if error timeout at their server
single_region_geometry_n = FisRequest(
    data_collection=DataCollection.SENTINEL5P,
    layer='NO2_SENTINEL5P',
    geometry_list=[single_region_geometry],
    time=('2020-06-20', '2020-12-31'),
    resolution='1000m',
    config=config
)

single_region_geometry_stats_n = single_region_geometry_n.get_data()

C:\Users\Administrator\anaconda3\lib\site-packages\sentinelhub\api\fis.py:97: SHDeprecationWarning: Fis service is being deprecated in favour of SentinelHubStatistical. Although no immediate action is needed as FIS is still supported, consider switching to Statistical API because it provides additional functionalities.
  warnings.warn(message, category=SHDeprecationWarning)


In [17]:
# request will return json output
single_region_geometry_stats_n

[{'C0': [{'date': '2020-12-31',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2020-12-30',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.46948389046132794,
     'stDev': 0.3963882794745913}},
   {'date': '2020-12-29',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.5655663656777348,
     'stDev': 0.3051516493130625}},
   {'date': '2020-12-28',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2020-12-27',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2020-12-26',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2020-12-25',
    'basicStats': {'min': 0.0, 'max': 0.0, 'mean': 0.0, 'stDev': 0.0}},
   {'date': '2020-12-24',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.4245163145508753,
     'stDev': 0.4042670376942354}},
   {'date': '2020-12-23',
    'basicStats'

In [19]:
# Data Frame Cleaning
single_region_stats_clean = clean_df(single_region_geometry_stats_n)
single_region_stats_clean

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2020-06-20,0.0,1.0,0.56211,0.303533,0.0,1.0,0.400873,0.457583
1,2020-06-21,0.0,1.0,0.426822,0.415694,0.0,1.0,0.553039,0.395934
2,2020-06-22,0.0,0.333102,0.061862,0.129535,0.0,1.0,0.479895,0.377552
3,2020-06-23,0.0,1.0,0.543417,0.221249,0.0,1.0,0.174826,0.343376
4,2020-06-24,0.0,1.0,0.660406,0.286693,0.0,1.0,0.458333,0.433572
...,...,...,...,...,...,...,...,...,...
190,2020-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,2020-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,2020-12-29,0.0,1.0,0.565566,0.305152,0.0,1.0,0.494036,0.462015
193,2020-12-30,0.0,1.0,0.469484,0.396388,0.0,1.0,0.585343,0.365885


In [ ]:
# Export to CSV
single_region_stats_clean.to_csv('regional_no2_results/ARMM_region_stats_clean.csv', index=False)

## END